In this notebook we perform experiments about the capabilities of LLMs of understanding the legality of moves in board games. 

We focus on the board game Chef's Hat (https://whisperproject.eu/chefshat). The rulebook is available at (https://github.com/pablovin/ChefsHatGYM/blob/master/gitImages/RulebookMenuv08.pdf).

In particular, the players have agency only on the first two phases of each round, and most of the decisions take in place during the second phase, which is the focus of this project work.

In this phase, players can either or play a hand of cards; the played cards must be played on top the cards already present on the board, and all of the cards must have the same value, which must be lower than the value of cards on the board. Moreover, it is not possible to play less cards than the amount of cards that are already on the board. Jokers can replace any card when played on the board, and any card can be put on top of them.

# Importing and cleaning data

We begin by creating our dataframe:

In [1]:
import pickle
import pandas as pd
import numpy as np
import torch
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig  

random_state = 1

with open('Dataset.pkl', 'rb') as f:
    data = pickle.load(f)

C:\Users\alfio\AppData\Local\Temp\ipykernel_17696\1749059438.py:12: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)


We inspect the attributes of the dataset; we will focus only on the "DISCARD" actions.

In [2]:
print(data.columns)
data

Index(['Match', 'Round', 'Agent_Names', 'Source', 'Action_Type',
       'Action_Description', 'Player_Finished', 'Player_Hands', 'Board_Before',
       'Board_After', 'Possible_Actions', 'Current_Roles', 'Match_Score',
       'Game_Score', 'Game_Performance_Score'],
      dtype='object')


,Match,Round,Agent_Names,Source,Action_Type,Action_Description,Player_Finished,Player_Hands,Board_Before,Board_After,Possible_Actions,Current_Roles,Match_Score,Game_Score,Game_Performance_Score
2024-11-21_15:03:51.314611,0,0,"[RANDOM_01, RANDOM_02, RANDOM_03, RANDOM_04]",SYSTEM,START_EXPERIMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-21_15:03:51.314611,1,0,NaN,SYSTEM,NEW_MATCH,NaN,NaN,NaN,NaN,NaN,NaN,"{'RANDOM_01': '', 'RANDOM_02': '', 'RANDOM_03'...",NaN,"{'RANDOM_01': -1, 'RANDOM_02': -1, 'RANDOM_03'...",NaN
2024-11-21_15:03:51.315609,1,0,NaN,SYSTEM,DEAL_CARDS,NaN,NaN,"[[3, 3, 5, 7, 7, 8, 8, 8, 9, 9, 9, 9, 0, 0, 0,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-21_15:03:51.327717,1,1,NaN,RANDOM_01,DISCARD,C11;Q4;J0,0.0,"[0, 0, 0, 0, 3, 3, 5, 7, 7, 8, 8, 8, 9, 9, 9, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]","[C11;Q1;J0, C11;Q1;J1, C11;Q2;J0, C11;Q2;J1, C...",NaN,NaN,NaN,NaN
2024-11-21_15:03:51.336602,1,1,NaN,RANDOM_02,DISCARD,pass,False,"[1, 4, 5, 5, 5, 6, 6, 7, 7, 7, 8, 9, 10, 10, 1...","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]",[pass],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-21_15:06:08.055500,100,26,NaN,RANDOM_01,DISCARD,pass,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C11;Q1;J0, pass]",NaN,NaN,NaN,NaN
2024-11-21_15:06:08.067496,100,26,NaN,RANDOM_02,DISCARD,C8;Q1;J0,True,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C8;Q1;J0, pass]",NaN,NaN,NaN,NaN
2024-11-21_15:06:08.070494,100,26,NaN,RANDOM_02,DECLARE_PIZZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-21_15:06:08.341109,100,26,NaN,SYSTEM,END_MATCH,NaN,NaN,NaN,NaN,NaN,NaN,"{'RANDOM_01': 'Dishwasher', 'RANDOM_02': 'Wait...","{'RANDOM_01': 0, 'RANDOM_02': 1, 'RANDOM_03': ...","{'RANDOM_01': 141, 'RANDOM_02': 150, 'RANDOM_0...",NaN


Selecting the relevant columns:

In [3]:
data_refactored = data[(data['Match'] > 0) & (data['Round'] > 0) & (data['Action_Type'] == 'DISCARD')][['Match', 'Action_Description', 'Player_Hands', 'Board_Before', 'Possible_Actions']]

data_refactored

,Match,Action_Description,Player_Hands,Board_Before,Possible_Actions
2024-11-21_15:03:51.327717,1,C11;Q4;J0,"[0, 0, 0, 0, 3, 3, 5, 7, 7, 8, 8, 8, 9, 9, 9, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C11;Q1;J0, C11;Q1;J1, C11;Q2;J0, C11;Q2;J1, C..."
2024-11-21_15:03:51.336602,1,pass,"[1, 4, 5, 5, 5, 6, 6, 7, 7, 7, 8, 9, 10, 10, 1...","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]",[pass]
2024-11-21_15:03:51.346670,1,pass,"[2, 4, 5, 6, 6, 7, 8, 8, 8, 8, 9, 9, 10, 10, 1...","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]","[C8;Q4;J0, pass]"
2024-11-21_15:03:51.356687,1,C10;Q4;J0,"[0, 0, 0, 0, 2, 3, 4, 4, 6, 6, 7, 9, 9, 10, 10...","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]","[C10;Q3;J1, C10;Q4;J0, C10;Q4;J1, C10;Q5;J0, C..."
2024-11-21_15:03:51.365671,1,C9;Q4;J0,"[0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 5, 7, 7, 8, 8, ...","[10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0]","[C8;Q3;J1, C9;Q3;J1, C9;Q4;J0, C9;Q4;J1, pass]"
...,...,...,...,...,...
2024-11-21_15:06:08.018273,100,pass,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[pass]
2024-11-21_15:06:08.028495,100,pass,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C8;Q1;J0, pass]"
2024-11-21_15:06:08.036495,100,C10;Q1;J0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C10;Q1;J0, C11;Q1;J0, pass]"
2024-11-21_15:06:08.055500,100,pass,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C11;Q1;J0, pass]"


In data_refactored, Player_Hands lists the cards that remain in hand AFTER the player has taken the action described in Action_Description.
 
We want to restore the correct player hand so that the Possible_actions column refers to the current player hand and board.

In [4]:
Action_Description_Refactored = data[(data['Match'] > 0) & (data['Round'] > 0) & (data['Action_Type'] == 'DISCARD')]['Action_Description']

New_Action_Description = []

for i in Action_Description_Refactored:
    if i != 'pass':
        actionlist = i.split(';')
    else:
        actionlist = i
    New_Action_Description.append(actionlist)

data_refactored['Action_Description'] = New_Action_Description


In [5]:
Player_Hand_Refactored = []

for i in range(len(data_refactored)):
        newel = list(data_refactored['Player_Hands'][i])#i-1?
        if data_refactored['Action_Description'][i] != 'pass':
                number = int(data_refactored['Action_Description'][i][0][1:])
                quantity = int(data_refactored['Action_Description'][i][1][1:])
                jolly = int(data_refactored['Action_Description'][i][2][1:])
                for k in range(quantity):
                        newel[newel.index(0)] = number
                for j in range(jolly):
                        newel[newel.index(0)] = 12
        newel.sort(reverse = False)
        newel = [int(i) for i in newel]
        Player_Hand_Refactored.append(newel)

data_refactored['Player_Hands'] = Player_Hand_Refactored
data_refactored

C:\Users\alfio\AppData\Local\Temp\ipykernel_17696\2413754936.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  newel = list(data_refactored['Player_Hands'][i])#i-1?
C:\Users\alfio\AppData\Local\Temp\ipykernel_17696\2413754936.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data_refactored['Action_Description'][i] != 'pass':
C:\Users\alfio\AppData\Local\Temp\ipykernel_17696\2413754936.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.i

,Match,Action_Description,Player_Hands,Board_Before,Possible_Actions
2024-11-21_15:03:51.327717,1,"[C11, Q4, J0]","[3, 3, 5, 7, 7, 8, 8, 8, 9, 9, 9, 9, 11, 11, 1...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C11;Q1;J0, C11;Q1;J1, C11;Q2;J0, C11;Q2;J1, C..."
2024-11-21_15:03:51.336602,1,pass,"[1, 4, 5, 5, 5, 6, 6, 7, 7, 7, 8, 9, 10, 10, 1...","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]",[pass]
2024-11-21_15:03:51.346670,1,pass,"[2, 4, 5, 6, 6, 7, 8, 8, 8, 8, 9, 9, 10, 10, 1...","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]","[C8;Q4;J0, pass]"
2024-11-21_15:03:51.356687,1,"[C10, Q4, J0]","[2, 3, 4, 4, 6, 6, 7, 9, 9, 10, 10, 10, 10, 10...","[11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0]","[C10;Q3;J1, C10;Q4;J0, C10;Q4;J1, C10;Q5;J0, C..."
2024-11-21_15:03:51.365671,1,"[C9, Q4, J0]","[0, 0, 0, 0, 3, 3, 5, 7, 7, 8, 8, 8, 9, 9, 9, ...","[10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0]","[C8;Q3;J1, C9;Q3;J1, C9;Q4;J0, C9;Q4;J1, pass]"
...,...,...,...,...,...
2024-11-21_15:06:08.018273,100,pass,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[pass]
2024-11-21_15:06:08.028495,100,pass,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C8;Q1;J0, pass]"
2024-11-21_15:06:08.036495,100,"[C10, Q1, J0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C10;Q1;J0, C11;Q1;J0, pass]"
2024-11-21_15:06:08.055500,100,pass,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[C11;Q1;J0, pass]"


We now select only the unique pairs of (player_hand, board_state):

In [6]:
data_refactored['Player_Hands'] = data_refactored['Player_Hands'].apply(tuple)
data_refactored['Board_Before'] = data_refactored['Board_Before'].apply(tuple)

data_refactored_unique = data_refactored.drop_duplicates(subset = ['Player_Hands', 'Board_Before'])
data_refactored_unique

,Match,Action_Description,Player_Hands,Board_Before,Possible_Actions
2024-11-21_15:03:51.327717,1,"[C11, Q4, J0]","(3, 3, 5, 7, 7, 8, 8, 8, 9, 9, 9, 9, 11, 11, 1...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C11;Q1;J0, C11;Q1;J1, C11;Q2;J0, C11;Q2;J1, C..."
2024-11-21_15:03:51.336602,1,pass,"(1, 4, 5, 5, 5, 6, 6, 7, 7, 7, 8, 9, 10, 10, 1...","(11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0)",[pass]
2024-11-21_15:03:51.346670,1,pass,"(2, 4, 5, 6, 6, 7, 8, 8, 8, 8, 9, 9, 10, 10, 1...","(11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0)","[C8;Q4;J0, pass]"
2024-11-21_15:03:51.356687,1,"[C10, Q4, J0]","(2, 3, 4, 4, 6, 6, 7, 9, 9, 10, 10, 10, 10, 10...","(11, 11, 11, 11, 0, 0, 0, 0, 0, 0, 0)","[C10;Q3;J1, C10;Q4;J0, C10;Q4;J1, C10;Q5;J0, C..."
2024-11-21_15:03:51.365671,1,"[C9, Q4, J0]","(0, 0, 0, 0, 3, 3, 5, 7, 7, 8, 8, 8, 9, 9, 9, ...","(10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0)","[C8;Q3;J1, C9;Q3;J1, C9;Q4;J0, C9;Q4;J1, pass]"
...,...,...,...,...,...
2024-11-21_15:06:07.788984,100,pass,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C7;Q1;J0, C7;Q2;J0, pass]"
2024-11-21_15:06:07.856173,100,pass,"(0, 0, 0, 0, 0, 0, 0, 8, 9, 10, 10, 10, 10, 11...","(7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]
2024-11-21_15:06:07.891175,100,"[C11, Q1, J0]","(0, 0, 0, 0, 0, 0, 0, 8, 9, 10, 10, 10, 10, 11...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C8;Q1;J0, C9;Q1;J0, C10;Q1;J0, C10;Q2;J0, C10..."
2024-11-21_15:06:07.904180,100,"[C11, Q2, J0]","(0, 0, 0, 0, 0, 0, 0, 0, 8, 9, 10, 10, 10, 10,...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C8;Q1;J0, C9;Q1;J0, C10;Q1;J0, C10;Q2;J0, C10..."


In [7]:
balanced_df_train = data_refactored_unique[(data_refactored_unique['Match'] <= 97)]
balanced_df_train_shuffled = balanced_df_train.sample(frac=1, random_state = random_state).reset_index(drop=True) #The other random_states in the .sample() methods shall be left fixed in order to guarantee that the test set remains the same
balanced_df_train_shuffled

,Match,Action_Description,Player_Hands,Board_Before,Possible_Actions
0,38,"[C3, Q1, J0]","(0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10...","(11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C1;Q1;J0, C2;Q1;J0, C3;Q1;J0, C4;Q1;J0, C5;Q1..."
1,11,"[C8, Q3, J0]","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, ...","(9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0)","[C8;Q3;J0, C8;Q4;J0, pass]"
2,80,pass,"(0, 0, 0, 0, 0, 0, 0, 0, 5, 8, 10, 10, 10, 11,...","(9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]
3,5,"[C11, Q2, J0]","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 9, 11, 11...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C5;Q1;J0, C9;Q1;J0, C11;Q1;J0, C11;Q2;J0, C11..."
4,10,"[C4, Q1, J0]","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 6, 8, 10,...","(5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C4;Q1;J0, pass]"
...,...,...,...,...,...
7593,11,pass,"(0, 0, 4, 6, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 1...","(12, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]
7594,65,pass,"(0, 2, 6, 6, 6, 6, 8, 8, 9, 10, 10, 10, 10, 10...","(3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C2;Q1;J0, pass]"
7595,49,"[C1, Q1, J0]","(0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 5, 6, 9, 10, 11...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C1;Q1;J0, C3;Q1;J0, C5;Q1;J0, C6;Q1;J0, C9;Q1..."
7596,3,pass,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10,...","(10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]


In [8]:
balanced_df_test = data_refactored_unique[(data_refactored_unique['Match'] > 97)]
balanced_df_test_shuffled = balanced_df_test.sample(frac=1, random_state = 42).reset_index(drop=True)
balanced_df_test_shuffled

,Match,Action_Description,Player_Hands,Board_Before,Possible_Actions
0,99,pass,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]
1,99,pass,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10...","(11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C10;Q2;J0, C10;Q3;J0, pass]"
2,99,pass,"(5, 5, 5, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, ...","(3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]
3,100,"[C9, Q3, J0]","(0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 8, 8, 9, 9, 9, ...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C6;Q1;J0, C8;Q1;J0, C8;Q2;J0, C9;Q1;J0, C9;Q2..."
4,98,"[C8, Q1, J0]","(0, 0, 3, 3, 4, 4, 4, 6, 6, 7, 7, 8, 9, 9, 10,...","(9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C3;Q1;J0, C3;Q2;J0, C4;Q1;J0, C4;Q2;J0, C4;Q3..."
...,...,...,...,...,...
216,99,pass,"(0, 0, 0, 0, 0, 6, 7, 8, 8, 9, 9, 9, 10, 10, 1...","(4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]
217,98,"[C9, Q1, J0]","(0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, ...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C5;Q1;J0, C5;Q2;J0, C5;Q3;J0, C6;Q1;J0, C6;Q2..."
218,99,"[C3, Q1, J0]","(0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 1...","(13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[C1;Q1;J0, C3;Q1;J0, C4;Q1;J0, C7;Q1;J0, C9;Q1..."
219,100,pass,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 1...","(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",[pass]


# Experiments

We perform experiments with the Llama-3.1-8B-Instruct model from Huggingface. We adopt 4-bit quantization for improving speed and performances. In all of the experiments we include three settings.

In the first case we do not provide any explanation about the semantics of the input and just ask the model to complete the query based on the provided examples.

In the second case we provide a basic explanation of the input, without references to the rules of the game.

Last, we give a full explanation about the semantics of the input, the rules of the game, and include known techniques of prompt engineering.

We evaluate results according to several metrics: all_moves_accuracy (the fraction of instances for which the full correct list of legal moves is returned), TP (the number of correctly identified legal moves), FP (the number of illegal moves returned), FN (the legal moves which are not returned by the model) and IoU_lists (a variant of Intersection over Union for lists)

In [9]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16,
                  "quantization_config": {"load_in_4bit": True}},
    device_map="cuda:0"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [10]:
def IoU_lists(s1: list[str], s2: list[str]) -> float:
    set1, set2 = set(s1), set(s2)
    return len(set1 & set2) / len(set1 | set2)

## Experiment 1: Random examples

The first experiment focuses on providing random examples to the model. Since we shuffled the dataset, we use the first 5/10/15 entries of the "train" set in order to select the examples.

### No explanations provided

First we provide only examples in the input, with no descriptions of its semantics

#### 5 examples

In [11]:
examples = ''
for i in range(5):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

query = 'A: '+ str(balanced_df_test_shuffled.iloc[0]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[0]['Board_Before'])+'\n'+'B: '

print('''Here's an example of input provided to Llama:\n\nComplete the following text. Only add the text that comes after "B:"''')
print(examples)
print(query)

Here's an example of input provided to Llama:

Complete the following text. Only add the text that comes after "B:"
A: (0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 11), (11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C1;Q1;J0', 'C2;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C5;Q1;J0', 'C6;Q1;J0', 'C7;Q1;J0', 'C8;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8), (9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C8;Q3;J0', 'C8;Q4;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 8, 10, 10, 10, 11, 11, 11, 11), (9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 9, 11, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C5;Q1;J0', 'C9;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 6, 8, 10, 10, 11), (5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C4;Q1;J0', 'pass']


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
#NO DESCRIPTION OF INPUT

len(balanced_df_test_shuffled) #.iloc[0]['Possible_Actions'] == final_output

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': 'Complete the following text. Only add the text that comes after "B:"'},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=500
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'C0;Q5;J0', 'C0;Q6;J0', 'C0;Q7;J0', 'C0;Q8;J0', 'C0;Q9;J0', 'C0;Q10;J0', 'C0;Q11;J0', 'C0;Q12;J0', 'C0;Q13;J0', 'C0;Q14;J0', 'C0;Q15;J0', 'C0;Q16;J0', 'C0;Q17;J0', 'C0;Q18;J0', 'C0;Q19;J0', 'C0;Q20;J0', 'C0;Q21;J0', 'C0;Q22;J0', 'C0;Q23;J0', 'C0;Q24;J0', 'C0;Q25;J0', 'C0;Q26;J0', 'C0;Q27;J0', 'C0;Q28;J0', 'C0;Q29;J0', 'C0;Q30;J0', 'C0;Q31;J0', 'C0;Q32;J0', 'C0;Q33;J0', 'C0;Q34;J0', 'C0;Q35;J0', 'C0;Q36;J0', 'C0;Q37;J0', 'C0;Q38;J0', 'C0;Q39;J0', 'C0;Q40;J0', 'C0;Q41;J0', 'C0;Q42;J0', 'C0;Q43;J0', 'C0;Q44;J0', 'C0;Q45;J0', 'C0;Q46;J0', 'C0;Q47;J0', 'C0;Q48;J0', 'C0;Q49;J0', 'C0;Q50;J0'] 
 [np.str_('pass')] 

TP: 0
FN: 1
FP: 50
IoU: 0.0 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C2;Q1;J0', 'C3;Q1;J0', 'C7;Q1;J0', 'C7;Q2;J0', 'C7;Q3;J0', 'C7;Q4;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 3
FN: 0
FP: 9
IoU: 0.25 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C5;Q3;J0', 'C6;Q1;J0', 'C6;Q2;J0', 'C6;Q3;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 10
FN: 0
FP: 3
IoU: 0.7692307692307693 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C7;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 12
FN: 0
FP: 5
IoU: 0.7058823529411765 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C1;Q1;J0', 'C2;Q1;J0', 'C3;Q1;J0', 'C6;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C10;Q5;J0', 'C10;Q6;J0', 'C10;Q7;J0', 'C10;Q8;J0', 'C10;Q9;J0', 'C10;Q10;J0', 'C10;Q11;J0', 'C10;Q12;J0', 'C10;Q13;J0', 'C10;Q14;J0', 'C10;Q15;J0', 'C10;Q16;J0', 'C10;Q17;J0', 'C10;Q18;J0', 'C10;Q19;J0', 'C10;Q20;J0', 'C10;Q21;J0', 'C10;Q22;J0', 'C10;Q23;J0', 'C10;Q24;J0', 'C10;Q25;J0', 'C10;Q26;J0', 'C10;Q27;J0', 'C10;Q28;J0', 'C10;Q29;J0', 'C10;Q30;J0', 'C10;Q31;J0', 'C10;Q32;J0', 'C10;Q33;J0', 'C10;Q34;J0', 'C10;Q35;J0', 'C10;Q36;J0', 'C10;Q37;J0', 'C10;Q38;J0', 'C10;Q39;J0', 'C10;Q40;J0', 'C10;Q41;J0', 'C10;Q42;J0', 'C10;Q43;J0', 'C10;Q44;J0'] 
 [np.str_('pass')] 

TP: 0
FN: 1
FP: 50
IoU: 0.0 


A: (0, 0, 0, 0, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C6;Q1;J0', 'C7;Q1;J0', 'C7;Q2;J0', 'C8;Q1;J0', 'C8;Q2;J0', 'C9;Q1;

#### 10 examples

In [13]:
examples = ''
for i in range(10):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

query = 'A: '+ str(balanced_df_test_shuffled.iloc[0]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[0]['Board_Before'])+'\n'+'B: '

print('''Here's an example of input provided to Llama:\n\nComplete the following text. Only add the text that comes after "B:"''')
print(examples)
print(query)

Here's an example of input provided to Llama:

Complete the following text. Only add the text that comes after "B:"
A: (0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 11), (11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C1;Q1;J0', 'C2;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C5;Q1;J0', 'C6;Q1;J0', 'C7;Q1;J0', 'C8;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8), (9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C8;Q3;J0', 'C8;Q4;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 8, 10, 10, 10, 11, 11, 11, 11), (9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 9, 11, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C5;Q1;J0', 'C9;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 6, 8, 10, 10, 11), (5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C4;Q1;J0', 'pass']

A: (2, 3, 4, 4, 5, 5, 6, 6, 6, 7, 8, 9, 9, 10, 10, 11, 11), (4, 4, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
#NO DESCRIPTION OF INPUT

len(balanced_df_test_shuffled) #.iloc[0]['Possible_Actions'] == final_output

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': 'Complete the following text. Only add the text that comes after "B:"'},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C9;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'C11;Q6;J0', 'C11;Q7;J0', 'C11;Q8;J0', 'C11;Q9;J0', 'C11;Q10;J0', 'C11;Q11;J0', 'C11;Q12;J0', 'C11;Q13;J0', 'C11;Q14;J0', 'C11;Q15;J0', 'C11;Q16;J0', 'C11;Q17;J0', 'C11;Q18;J0', 'C11;Q19;J0', 'C11;Q20;J0', 'C11;Q21;J0', 'C11;Q22;J0', 'C11;Q23;J0', 'C11;Q24;J0', 'C11;Q25;J0', 'C11;Q26;J0', 'C11;Q27;J0', 'C11;Q28;J0', 'C11;Q29;J0', 'C11;Q30;J0', 'C11;Q31;J0', 'C11;Q32;J0', 'C11;Q33;J0', 'C11;Q34;J0', 'C11;Q35;J0', 'C11;Q36;J0', 'C11;Q37;J0', 'C11;Q38;J0', 'C11;Q39;J0', 'C11;Q40;J0', 'C11;Q41;J0', 'C11;Q42;J0', 'C11;Q43;J0', 'C11;Q44;J0', 'C11;Q45;J0', 'C11;Q46;J0', 'C11;Q47;J0', 'C11;Q48;J0', 'C11;Q49;J0', 'C11;Q50;J0', 'C11;Q51;J0', 'C11;Q52;J0', 'C11;Q53;J0', 'C11;Q54;J0', 'C11;Q55;J0', 'C11;Q56;J0', 'C11;Q57;J0', 'C11;Q58;J0', 'C11;Q59;J0', 'C11;Q60;J0', 'C11;Q61;J0', 'C11;Q62;J0', 'C11;Q63;J0', 'C11;Q64;J0', 'C11;

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C2;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C5;Q1;J0', 'C6;Q1;J0', 'C7;Q1;J0', 'C8;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 3
FN: 0
FP: 10
IoU: 0.23076923076923078 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C5;Q3;J0', 'C5;Q4;J0', 'C5;Q5;J0', 'C5;Q6;J0', 'C5;Q7;J0', 'C5;Q8;J0', 'C5;Q9;J0', 'C5;Q10;J0', 'C5;Q11;J0', 'C5;Q12;J0', 'C5;Q13;J0', 'C5;Q14;J0', 'C5;Q15;J0', 'C5;Q16;J0', 'C5;Q17;J0', 'C5;Q18;J0', 'C5;Q19;J0', 'C5;Q20;J0', 'C5;Q21;J0', 'C5;Q22;J0', 'C5;Q23;J0', 'C5;Q24;J0', 'C5;Q25;J0', 'C5;Q26;J0', 'C5;Q27;J0', 'C5;Q28;J0', 'C5;Q29;J0', 'C5;Q30;J0', 'C5;Q31;J0', 'C5;Q32;J0', 'C5;Q33;J0', 'C5;Q34;J0', 'C5;Q35;J0', 'C5;Q36;J0', 'C5;Q37;J0', 'C5;Q38;J0', 'C5;Q39;J0', 'C5;Q40;J0', 'C5;Q41;J0', 'C5;Q42;J0', 'C5;Q43;J0', 'C5;Q44;J0', 'C5;Q45;J0', 'C5;Q46;J0', 'C5;Q47;J0', 'C5;Q48;J0', 'C5;Q49;J0', 'C5;Q50;J0', 'C5;Q51;J0', 'C5;Q52;J0', 'C5;Q53;J0', 'C5;Q54;J0', 'C5;Q55;J0', 'C5;Q56;J0', 'C5;Q57;J0', 'C5;Q58;J0', 'C5;Q59;J0', 'C5;Q60;J0', 'C5;Q61;J0', 'C5;Q62;J0', 'C5;Q63;J0', 'C5;Q64;J0', 'C5;Q65;J0', 'C5;Q66;J0', 'C5;Q67;J0', 'C5;Q68;J0', 'C5;Q69;J0', 'C5;Q70;J0', 'C

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C7;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 7
FN: 5
FP: 3
IoU: 0.4666666666666667 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'C0;Q5;J0', 'C0;Q6;J0', 'C0;Q7;J0', 'C0;Q8;J0', 'C0;Q9;J0', 'C0;Q10;J0', 'C0;Q11;J0', 'C0;Q12;J0', 'C0;Q13;J0', 'C0;Q14;J0', 'C0;Q15;J0', 'C0;Q16;J0', 'C0;Q17;J0', 'C0;Q18;J0', 'C0;Q19;J0', 'C0;Q20;J0', 'C0;Q21;J0', 'C0;Q22;J0', 'C0;Q23;J0', 'C0;Q24;J0', 'C0;Q25;J0', 'C0;Q26;J0', 'C0;Q27;J0', 'C0;Q28;J0', 'C0;Q29;J0', 'C0;Q30;J0', 'C0;Q31;J0', 'C0;Q32;J0', 'C0;Q33;J0', 'C0;Q34;J0', 'C0;Q35;J0', 'C0;Q36;J0', 'C0;Q37;J0', 'C0;Q38;J0', 'C0;Q39;J0', 'C0;Q40;J0', 'C0;Q41;J0', 'C0;Q42;J0', 'C0;Q43;J0', 'C0;Q44;J0', 'C0;Q45;J0', 'C0;Q46;J0', 'C0;Q47;J0', 'C0;Q48;J0', 'C0;Q49;J0', 'C0;Q50;J0', 'C0;Q51;J0', 'C0;Q52;J0', 'C0;Q53;J0', 'C0;Q54;J0', 'C0;Q55;J0', 'C0;Q56;J0', 'C0;Q57;J0', 'C0;Q58;J0', 'C0;Q59;J0', 'C0;Q60;J0', 'C0;Q61;J0', 'C0;Q62;J0', 'C0;Q63;J0', 'C0;Q64;J0', 'C0;Q65;J0', 'C0;Q66;J0', 'C0;Q67;J0', 'C0;Q68;J0', 'C0;Q69;J0', 'C0;Q70;J0',

#### 15 examples

In [15]:
examples = ''
for i in range(15):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

query = 'A: '+ str(balanced_df_test_shuffled.iloc[0]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[0]['Board_Before'])+'\n'+'B: '

print('''Here's an example of input provided to Llama:\n\nComplete the following text. Only add the text that comes after "B:"''')
print(examples)
print(query)

Here's an example of input provided to Llama:

Complete the following text. Only add the text that comes after "B:"
A: (0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 11), (11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C1;Q1;J0', 'C2;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C5;Q1;J0', 'C6;Q1;J0', 'C7;Q1;J0', 'C8;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8), (9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C8;Q3;J0', 'C8;Q4;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 8, 10, 10, 10, 11, 11, 11, 11), (9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 9, 11, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C5;Q1;J0', 'C9;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass']

A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 6, 8, 10, 10, 11), (5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: ['C4;Q1;J0', 'pass']

A: (2, 3, 4, 4, 5, 5, 6, 6, 6, 7, 8, 9, 9, 10, 10, 11, 11), (4, 4, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
#NO DESCRIPTION OF INPUT

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': 'Complete the following text. Only add the text that comes after "B:"'},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'C0;Q5;J0', 'C0;Q6;J0', 'C0;Q7;J0', 'C0;Q8;J0', 'C0;Q9;J0', 'C0;Q10;J0', 'C0;Q11;J0', 'C0;Q12;J0', 'C0;Q13;J0', 'C0;Q14;J0', 'C0;Q15;J0', 'C0;Q16;J0', 'C0;Q17;J0', 'C0;Q18;J0', 'C0;Q19;J0', 'C0;Q20;J0', 'C0;Q21;J0', 'C0;Q22;J0', 'C0;Q23;J0', 'C0;Q24;J0', 'C0;Q25;J0', 'C0;Q26;J0', 'C0;Q27;J0', 'C0;Q28;J0', 'C0;Q29;J0', 'C0;Q30;J0', 'C0;Q31;J0', 'C0;Q32;J0', 'C0;Q33;J0', 'C0;Q34;J0', 'C0;Q35;J0', 'C0;Q36;J0', 'C0;Q37;J0', 'C0;Q38;J0', 'C0;Q39;J0', 'C0;Q40;J0', 'C0;Q41;J0', 'C0;Q42;J0', 'C0;Q43;J0', 'C0;Q44;J0', 'C0;Q45;J0', 'C0;Q46;J0', 'C0;Q47;J0', 'C0;Q48;J0', 'C0;Q49;J0', 'C0;Q50;J0', 'C0;Q51;J0', 'C0;Q52;J0', 'C0;Q53;J0', 'C0;Q54;J0', 'C0;Q55;J0', 'C0;Q56;J0', 'C0;Q57;J0', 'C0;Q58;J0', 'C0;Q59;J0', 'C0;Q60;J0', 'C0;Q61;J0', 'C0;Q62;J0', 'C0;Q63;J0', 'C0;Q64;J0', 'C0;Q65;J0', 'C0;Q66;J0', 'C0;Q67;J0', 'C0;Q68;J0', 'C0;Q69;J0', 'C0;Q70;J0', 'C0

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 3
FN: 0
FP: 7
IoU: 0.3 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C6;Q1;J0', 'C6;Q2;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C10;Q1;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 9
FN: 1
FP: 2
IoU: 0.75 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C7;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C10;Q5;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 12
FN: 0
FP: 8
IoU: 0.6 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C3;Q1;J0', 'C6;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 11
IoU: 0.08333333333333333 


A: (0, 0, 0, 0, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C6;Q1;J0', 'C7;Q1;J0', 'C7;Q2;J0', 'C8;Q1;J0', 'C8;Q2;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C9;Q6;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C10;Q5;J0', 'C10;Q6;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'C11;Q6;J0', 'pass'] 
 [np.str_('C6;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C7;Q2;J0'), np.str_('C8;Q1;J0'), np.str_('C8;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C11;Q1;J0'), np.str_('C11;Q2;J0'), np.str_('C11;Q3;J0'), np.st

### Minimal description

#### 5 examples

In [17]:
examples = ''
for i in range(5):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

In [18]:
#MINIMAL DESCRIPTION OF INPUT

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': '''In the following examples "A" provides two lists. The first list is a description of the cards in hand of a bord game player. The second list is a description of the cards that are already on the board. In both cases a "0" represents a missing card. Then, "B" replies with the full list of the current legal moves according to the description of "A". Complete the text by listing the correct legal moves given the last description from "A". Only add the text that comes after "B:"
'''},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C1;Q1;J0', 'C9;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 7
IoU: 0.125 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 3
FN: 0
FP: 6
IoU: 0.3333333333333333 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C5;Q3;J0', 'C5;Q4;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 8
FN: 2
FP: 5
IoU: 0.5333333333333333 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C1;Q2;J0', 'C1;Q3;J0', 'C1;Q4;J0', 'C3;Q1;J0', 'C3;Q2;J0', 'C3;Q3;J0', 'C3;Q4;J0', 'C4;Q1;J0', 'C4;Q2;J0', 'C4;Q3;J0', 'C4;Q4;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 4
FN: 8
FP: 9
IoU: 0.19047619047619047 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'C3;Q1;J0', 'C3;Q2;J0', 'C3;Q3;J0', 'C3;Q4;J0', 'C6;Q1;J0', 'C6;Q2;J0', 'C6;Q3;J0', 'C6;Q4;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 24
IoU: 0.04 


A: (0, 0, 0, 0, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C6;Q1;J0', 'C7;Q1;J0', 'C7;Q2;J0', 'C8;Q1;J0', 'C8;Q2;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('C6;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C7;Q2;J0'), np.str_('C8;Q1;J0'), np.str_('C8;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C11;Q1;J0'), np.str_('

#### 10 examples

In [19]:
examples = ''
for i in range(10):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

In [20]:
#MINIMAL DESCRIPTION OF INPUT

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': '''In the following examples "A" provides two lists. The first list is a description of the cards in hand of a bord game player. The second list is a description of the cards that are already on the board. In both cases a "0" represents a missing card. Then, "B" replies with the full list of the current legal moves according to the description of "A". Complete the text by listing the correct legal moves given the last description from "A". Only add the text that comes after "B:"
'''},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C9;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 6
IoU: 0.14285714285714285 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 3
FN: 0
FP: 6
IoU: 0.3333333333333333 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C5;Q3;J0', 'C5;Q4;J0', 'C5;Q5;J0', 'C6;Q1;J0', 'C6;Q2;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 9
FN: 1
FP: 4
IoU: 0.6428571428571429 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C7;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 7
FN: 5
FP: 3
IoU: 0.4666666666666667 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'C0;Q5;J0', 'C0;Q6;J0', 'C0;Q7;J0', 'C0;Q8;J0', 'C0;Q9;J0', 'C0;Q10;J0', 'C0;Q11;J0', 'C0;Q12;J0', 'C0;Q13;J0', 'C0;Q14;J0', 'C0;Q15;J0', 'C0;Q16;J0', 'C0;Q17;J0', 'C0;Q18;J0', 'C0;Q19;J0', 'C0;Q20;J0', 'C0;Q21;J0', 'C0;Q22;J0', 'C0;Q23;J0', 'C0;Q24;J0', 'C0;Q25;J0', 'C0;Q26;J0', 'C0;Q27;J0', 'C0;Q28;J0', 'C0;Q29;J0', 'C0;Q30;J0', 'C0;Q31;J0', 'C0;Q32;J0', 'C0;Q33;J0', 'C0;Q34;J0', 'C0;Q35;J0', 'C0;Q36;J0', 'C0;Q37;J0', 'C0;Q38;J0', 'C0;Q39;J0', 'C0;Q40;J0', 'C0;Q41;J0', 'C0;Q42;J0', 'C0;Q43;J0', 'C0;Q44;J0', 'C0;Q45;J0', 'C0;Q46;J0', 'C0;Q47;J0', 'C0;Q48;J0', 'C0;Q49;J0', 'C0;Q50;J0', 'C0;Q51;J0', 'C0;Q52;J0', 'C0;Q53;J0', 'C0;Q54;J0', 'C0;Q55;J0', 'C0;Q56;J0', 'C0;Q57;J0', 'C0;Q58;J0', 'C0;Q59;J0', 'C0;Q60;J0', 'C0;Q61;J0', 'C0;Q62;J0', 'C0;Q63;J0', 'C0;Q64;J0', 'C0;Q65;J0', 'C0;Q66;J0', 'C0;Q67;J0', 'C0;Q68;J0', 'C0;Q69;J0', 'C0;Q70;J0',

#### 15 examples

In [21]:
examples = ''
for i in range(15):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

In [22]:
#MINIMAL DESCRIPTION OF INPUT

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': '''In the following examples "A" provides two lists. The first list is a description of the cards in hand of a bord game player. The second list is a description of the cards that are already on the board. In both cases a "0" represents a missing card. Then, "B" replies with the full list of the current legal moves according to the description of "A". Complete the text by listing the correct legal moves given the last description from "A". Only add the text that comes after "B:"
'''},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C9;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 6
IoU: 0.14285714285714285 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 3
FN: 0
FP: 1
IoU: 0.75 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C6;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 6
FN: 4
FP: 2
IoU: 0.5 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C1;Q2;J0', 'C1;Q3;J0', 'C1;Q4;J0', 'C1;Q5;J0', 'C1;Q6;J0', 'C1;Q7;J0', 'C1;Q8;J0', 'C1;Q9;J0', 'C1;Q10;J0', 'C1;Q11;J0', 'C1;Q12;J0', 'C1;Q13;J0', 'C3;Q1;J0', 'C3;Q2;J0', 'C3;Q3;J0', 'C3;Q4;J0', 'C3;Q5;J0', 'C3;Q6;J0', 'C3;Q7;J0', 'C3;Q8;J0', 'C3;Q9;J0', 'C3;Q10;J0', 'C3;Q11;J0', 'C3;Q12;J0', 'C3;Q13;J0', 'C4;Q1;J0', 'C4;Q2;J0', 'C4;Q3;J0', 'C4;Q4;J0', 'C4;Q5;J0', 'C4;Q6;J0', 'C4;Q7;J0', 'C4;Q8;J0', 'C4;Q9;J0', 'C4;Q10;J0', 'C4;Q11;J0', 'C4;Q12;J0', 'C4;Q13;J0', 'C7;Q1;J0', 'C7;Q2;J0', 'C7;Q3;J0', 'C7;Q4;J0', 'C7;Q5;J0', 'C7;Q6;J0', 'C7;Q7;J0', 'C7;Q8;J0', 'C7;Q9;J0', 'C7;Q10;J0', 'C7;Q11;J0', 'C7;Q12;J0', 'C7;Q13;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C9;Q6;J0', 'C9;Q7;J0', 'C9;Q8;J0', 'C9;Q9;J0', 'C9;Q10;J0', 'C9;Q11;J0', 'C9;Q12;J0', 'C9;Q13;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C10;Q5;J0', 'C10;Q6;J0', 'C10;Q7;J0', 'C10;Q8;

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C3;Q1;J0', 'C6;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 8
IoU: 0.1111111111111111 


A: (0, 0, 0, 0, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C6;Q1;J0', 'C7;Q1;J0', 'C7;Q2;J0', 'C8;Q1;J0', 'C8;Q2;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C10;Q5;J0', 'C10;Q6;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'C11;Q6;J0', 'pass'] 
 [np.str_('C6;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C7;Q2;J0'), np.str_('C8;Q1;J0'), np.str_('C8;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C11;Q1;J0'), np.str_('C11;Q2;J0'), np.str_('C11;Q3;J0'), np.str_('pass')] 

TP: 14
FN: 0
FP: 9
IoU: 0.6086956521739

### Full description

#### 5 examples

In [23]:
examples = ''
for i in range(5):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

In [24]:
#FULL DESCRIPTION OF INPUT

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': '''You are an expert board game player, and you are playing a new card game called Chef\'s Hat. In the following examples "A" provides two lists. The first list is a description of the cards in hand of a player. The second list is a description of the cards that are already on the board. In both cases a "0" represents a missing card, while a "12" represents a Joker card. "B" replies with the full list of the current legal moves according to the description of "A". The move 'CX;QY;JZ' means that Y cards with value X are played, while JZ indicates that Z Jokers are played.\n
                 As a player, you must play cards on top of all the cards that have been placed on the board before. The cards played must all have the same value, and this value must be strictly lower than the value of cards currently on the board. You cannot place less cards on the board than the amount of cards currently present there. If you have Joker cards, you can use them as if they had any value.\n
Complete the text by listing all the current legal moves given the last description from "A". Only add the text that comes after "B:"'''},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C1;Q2;J0', 'C1;Q3;J0', 'C1;Q4;J0', 'C1;Q5;J0', 'C1;Q6;J0', 'C1;Q7;J0', 'C1;Q8;J0', 'C1;Q9;J0', 'C1;Q10;J0', 'C1;Q11;J0', 'C1;Q12;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C9;Q6;J0', 'C9;Q7;J0', 'C9;Q8;J0', 'C9;Q9;J0', 'C9;Q10;J0', 'C9;Q11;J0', 'C9;Q12;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 24
IoU: 0.04 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'C0;Q5;J0', 'C0;Q6;J0', 'C0;Q7;J0', 'C0;Q8;J0', 'C0;Q9;J0', 'C0;Q10;J0', 'C0;Q11;J0', 'C0;Q12;J0', 'C0;Q13;J0', 'C0;Q14;J0', 'C0;Q15;J0', 'C0;Q16;J0', 'C0;Q17;J0', 'C0;Q18;J0', 'C0;Q19;J0', 'C0;Q20;J0', 'C0;Q21;J0', 'C0;Q22;J0', 'C0;Q23;J0', 'C0;Q24;J0', 'C0;Q25;J0', 'C0;Q26;J0', 'C0;Q27;J0', 'C0;Q28;J0', 'C0;Q29;J0', 'C0;Q30;J0', 'C0;Q31;J0', 'C0;Q32;J0', 'C0;Q33;J0', 'C0;Q34;J0', 'C0;Q35;J0', 'C0;Q36;J0', 'C0;Q37;J0', 'C0;Q38;J0', 'C0;Q39;J0', 'C0;Q40;J0', 'C0;Q41;J0', 'C0;Q42;J0', 'C0;Q43;J0', 'C0;Q44;J0', 'C0;Q45;J0', 'C0;Q46;J0', 'C0;Q47;J0', 'C0;Q48;J0', 'C0;Q49;J0', 'C0;Q50;J0', 'C0;Q51;J0', 'C0;Q52;J0', 'C0;Q53;J0', 'C0;Q54;J0', 'C0;Q55;J0', 'C0;Q56;J0', 'C0;Q57;J0', 'C0;Q58;J0', 'C0;Q59;J0', 'C0;Q60;J0', 'C0;Q61;J0', 'C0;Q62;J0', 'C0;Q63;J0', 'C0;Q64;J0', 'C0;Q65;J0', 'C0;Q66;J0', 'C0;Q67;J0', 'C0;Q68;J0', 'C0;Q69;J0', 'C0;Q70;J0'

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C5;Q3;J0', 'C5;Q4;J0', 'C5;Q5;J0', 'C6;Q1;J0', 'C6;Q2;J0', 'C6;Q3;J0', 'C6;Q4;J0', 'C6;Q5;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C10;Q5;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 10
FN: 0
FP: 11
IoU: 0.47619047619047616 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C7;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C10;Q5;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 12
FN: 0
FP: 4
IoU: 0.75 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C0;Q1;J0', 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 4
IoU: 0.2 


A: (0, 0, 0, 0, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C6;Q1;J0', 'C7;Q1;J0', 'C8;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('C6;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C7;Q2;J0'), np.str_('C8;Q1;J0'), np.str_('C8;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C11;Q1;J0'), np.str_('C11;Q2;J0'), np.str_('C11;Q3;J0'), np.str_('pass')] 

TP: 9
FN: 5
FP: 1
IoU: 0.6 


all_moves_accuracy: 0.03167420814479638
Avg. IoU: 0.2690723311690847
TP total: 688
FP total: 3192
FN total: 273


#### 10 examples

In [25]:
examples = ''
for i in range(10):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

In [26]:
#FULL DESCRIPTION OF INPUT

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': '''You are an expert board game player, and you are playing a new card game called Chef\'s Hat. In the following examples "A" provides two lists. The first list is a description of the cards in hand of a player. The second list is a description of the cards that are already on the board. In both cases a "0" represents a missing card, while a "12" represents a Joker card. "B" replies with the full list of the current legal moves according to the description of "A". The move 'CX;QY;JZ' means that Y cards with value X are played, while JZ indicates that Z Jokers are played.\n
                 As a player, you must play cards on top of all the cards that have been placed on the board before. The cards played must all have the same value, and this value must be strictly lower than the value of cards currently on the board. You cannot place less cards on the board than the amount of cards currently present there. If you have Joker cards, you can use them as if they had any value.\n
Complete the text by listing all the current legal moves given the last description from "A". Only add the text that comes after "B:"'''},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
[" ['C9;Q1;J0", 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 9
IoU: 0.1 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
[" ['C0;Q1;J0", 'C0;Q2;J0', 'C0;Q3;J0', 'C0;Q4;J0', 'C0;Q5;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 1
FN: 2
FP: 5
IoU: 0.125 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C5;Q3;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 4
FN: 6
FP: 0
IoU: 0.4 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C7;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 7
FN: 5
FP: 3
IoU: 0.4666666666666667 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
[' (', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '1', '', '', '', '', '', '', '', '', '', '', ")\nB: ['C3;Q1;J0", 'C6;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 11
IoU: 0.08333333333333333 


A: (0, 0, 0, 0, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
[' (', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '1', '1', '', '', '', '', '', '', '', '', '', ")\nB: ['C6;Q1;J0", 'C6;Q2;J0', 'C7;Q1;J0', 'C8;Q1;J0', 'C8;Q2;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'pass'] 
 [np.str_('C6;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C7;Q2;J0'), np.str_('C8;Q1;J0'), np.str_('C8;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0

#### 15 examples

In [27]:
examples = ''
for i in range(15):
    examples += 'A: '+ str(balanced_df_train_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_train_shuffled.iloc[i]['Board_Before'])+'\n'
    examples += 'B: '+ str([str(el) for el in balanced_df_train_shuffled.iloc[i]['Possible_Actions']])+'\n\n'

In [28]:
#FULL DESCRIPTION OF INPUT

count = 0
IoU = 0
TP = 0
FN = 0
FP = 0

for i in range(len(balanced_df_test_shuffled)):
    query = 'A: '+ str(balanced_df_test_shuffled.iloc[i]['Player_Hands'])+', '+str(balanced_df_test_shuffled.iloc[i]['Board_Before'])+'\n'+'B: '
    messages = [{'role': 'system', 'content': '''You are an expert board game player, and you are playing a new card game called Chef\'s Hat. In the following examples "A" provides two lists. The first list is a description of the cards in hand of a player. The second list is a description of the cards that are already on the board. In both cases a "0" represents a missing card, while a "12" represents a Joker card. "B" replies with the full list of the current legal moves according to the description of "A". The move 'CX;QY;JZ' means that Y cards with value X are played, while JZ indicates that Z Jokers are played.\n
                 As a player, you must play cards on top of all the cards that have been placed on the board before. The cards played must all have the same value, and this value must be strictly lower than the value of cards currently on the board. You cannot place less cards on the board than the amount of cards currently present there. If you have Joker cards, you can use them as if they had any value.\n
Complete the text by listing all the current legal moves given the last description from "A". Only add the text that comes after "B:"'''},{"role": "system", "content": examples},{"role": "user", "content": query}]
    outputs = pipeline(
        messages,
        #max_length=1053,
        max_new_tokens=1000
    )
    ref_output = outputs[0]['generated_text'][-1]['content'][1:-1].split(sep = ', ')
    final_output = [elem[1:-1] for elem in ref_output]
    print(query)
    print(final_output,'\n',balanced_df_test_shuffled.iloc[i]['Possible_Actions'],'\n')

    if final_output == balanced_df_test_shuffled.iloc[i]['Possible_Actions']:
        count += 1

    TP_temp = len(set(final_output) & set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    TP += TP_temp
    print('TP:', TP_temp)

    FN_temp = len(set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']) - set(final_output))
    FN += FN_temp
    print('FN:', FN_temp)

    FP_temp = len(set(final_output) - set(balanced_df_test_shuffled.iloc[i]['Possible_Actions']))
    FP += FP_temp
    print('FP:', FP_temp)

    IoU_serv = IoU_lists(final_output, balanced_df_test_shuffled.iloc[i]['Possible_Actions'])   #IoU
    IoU += IoU_serv
    print('IoU:', IoU_serv,'\n\n')


print('all_moves_accuracy:', count/len(balanced_df_test_shuffled))
print('Avg. IoU:', IoU/len(balanced_df_test_shuffled))
print('TP total:', TP)
print('FP total:', FP)
print('FN total:', FN)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C9;Q6;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'C11;Q3;J0', 'C11;Q4;J0', 'C11;Q5;J0', 'C11;Q6;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 12
IoU: 0.07692307692307693 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 11), (11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'pass'] 
 [np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('pass')] 

TP: 3
FN: 0
FP: 1
IoU: 0.75 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

A: (0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 6, 6, 9, 9, 9, 10), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C5;Q1;J0', 'C5;Q2;J0', 'C5;Q3;J0', 'C5;Q4;J0', 'C5;Q5;J0', 'C5;Q6;J0', 'C5;Q7;J0', 'C5;Q8;J0', 'C5;Q9;J0', 'C5;Q10;J0', 'C6;Q1;J0', 'C6;Q2;J0', 'C6;Q3;J0', 'C6;Q4;J0', 'C6;Q5;J0', 'C6;Q6;J0', 'C6;Q7;J0', 'C6;Q8;J0', 'C6;Q9;J0', 'C6;Q10;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'C9;Q6;J0', 'C9;Q7;J0', 'C9;Q8;J0', 'C9;Q9;J0', 'C9;Q10;J0', 'C10;Q1;J0', 'pass'] 
 [np.str_('C5;Q1;J0'), np.str_('C5;Q2;J0'), np.str_('C5;Q3;J0'), np.str_('C6;Q1;J0'), np.str_('C6;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('pass')] 

TP: 10
FN: 0
FP: 22
IoU: 0.3125 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 1, 3, 4, 7, 9, 9, 10, 10, 10, 10, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C1;Q1;J0', 'C3;Q1;J0', 'C4;Q1;J0', 'C7;Q1;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C10;Q4;J0', 'C11;Q1;J0', 'pass'] 
 [np.str_('C1;Q1;J0'), np.str_('C3;Q1;J0'), np.str_('C4;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C10;Q3;J0'), np.str_('C10;Q4;J0'), np.str_('C11;Q1;J0'), np.str_('pass')] 

TP: 12
FN: 0
FP: 0
IoU: 1.0 




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: (0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 9, 10, 10, 10, 11, 11), (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C3;Q1;J0', 'C6;Q1;J0', 'C9;Q1;J0', 'C10;Q1;J0', 'C10;Q2;J0', 'C10;Q3;J0', 'C11;Q1;J0', 'C11;Q2;J0', 'pass'] 
 [np.str_('pass')] 

TP: 1
FN: 0
FP: 8
IoU: 0.1111111111111111 


A: (0, 0, 0, 0, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 11), (13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
B: 
['C6;Q1;J0', 'C7;Q1;J0', 'C7;Q2;J0', 'C8;Q1;J0', 'C8;Q2;J0', 'C9;Q1;J0', 'C9;Q2;J0', 'C9;Q3;J0', 'C9;Q4;J0', 'C9;Q5;J0', 'pass'] 
 [np.str_('C6;Q1;J0'), np.str_('C7;Q1;J0'), np.str_('C7;Q2;J0'), np.str_('C8;Q1;J0'), np.str_('C8;Q2;J0'), np.str_('C9;Q1;J0'), np.str_('C9;Q2;J0'), np.str_('C9;Q3;J0'), np.str_('C10;Q1;J0'), np.str_('C10;Q2;J0'), np.str_('C11;Q1;J0'), np.str_('C11;Q2;J0'), np.str_('C11;Q3;J0'), np.str_('pass')] 

TP: 9
FN: 5
FP: 2
IoU: 0.5625 


all_moves_accuracy: 0.02262443438914027
Avg. IoU: 0.26389219199168773
TP total: 819
FP total: 3109
FN total: 142
